# Lesson Plan
In this session, we will be working with the Udacity student data to master key concepts in the data exploration phase:
    - Data Ingestion
    - Data Cleaning
    - Sanity Checking
    - Describing the Data
    - Slicing Data
    - Grouping Data
    - Crunching and Validating Numbers
    - Plotting

# Part 1: Data Ingestion and Wrangling

# Ingesting CSVs

### Ingesting CSVs using your own code
First think about what data structure to read in. Here, I am going to read in a list of dictionaries, with each record being a dictionary.

In [1]:
keys = ["account_key",
        "status",
        "join_date",
        "cancel_date",
        "days_to_cancel",
        "is_udacity",
        "is_canceled"]

def read_csv(filename):
    data = list()
    with open(filename) as f:
        for line in f:
            parts = line.strip().split(",")
            data.append(dict(zip(keys, parts)))
    return data

enrollments = read_csv("enrollments.csv")
print(enrollments[0])

daily_engagement = read_csv("daily_engagement.csv")
project_submissions = read_csv("project_submissions.csv")


{'account_key': 'account_key', 'status': 'status', 'join_date': 'join_date', 'cancel_date': 'cancel_date', 'days_to_cancel': 'days_to_cancel', 'is_udacity': 'is_udacity', 'is_canceled': 'is_canceled'}


* Observation: But what if we don't know the header fields? What if the header fields change? We should avoid hard-coding as much as possible. 
* <b> Better version </b>: The following piece of code uses the csv library which provides a function that interprets the first row as the header, and populates the list of dictionaries accordingly.


### Ingesting CSVs using the csv library

In [2]:
import csv

def read_csv(filename):
    with open(filename) as csvfile:
        data = csv.DictReader(csvfile)
        return list(data)
            
enrollments = read_csv("enrollments.csv")
daily_engagement = read_csv("daily_engagement.csv")
project_submissions = read_csv("project_submissions.csv")
print(enrollments[0])

OrderedDict([('account_key', '448'), ('status', 'canceled'), ('join_date', '2014-11-10'), ('cancel_date', '2015-01-14'), ('days_to_cancel', '65'), ('is_udacity', 'True'), ('is_canceled', 'True')])


### Ingesting CSVs using the Pandas library
We will learn about Pandas and the data-types it offers later. But for now let us see how ingestion looks, and how Pandas significantly makes code cleaner and the output much more readable. It also makes ingestion faster!


In [3]:
import pandas as pd

enrollments_pd = pd.read_csv("enrollments.csv")
daily_engagement_pd = pd.read_csv("daily_engagement.csv")
project_submissions_pd = pd.read_csv("project_submissions.csv")

enrollments_pd.head()

,account_key,status,join_date,cancel_date,days_to_cancel,is_udacity,is_canceled
0,448,canceled,2014-11-10,2015-01-14,65.0,True,True
1,448,canceled,2014-11-05,2014-11-10,5.0,True,True
2,448,canceled,2015-01-27,2015-01-27,0.0,True,True
3,448,canceled,2014-11-10,2014-11-10,0.0,True,True
4,448,current,2015-03-10,NaN,NaN,True,False


In [4]:
project_submissions_pd.head()

,creation_date,completion_date,assigned_rating,account_key,lesson_key,processing_state
0,2015-01-14,2015-01-16,UNGRADED,256,3176718735,EVALUATED
1,2015-01-10,2015-01-13,INCOMPLETE,256,3176718735,EVALUATED
2,2015-01-20,2015-01-20,PASSED,256,3176718735,EVALUATED
3,2015-03-10,2015-03-13,PASSED,434,3176718735,EVALUATED
4,2015-02-17,2015-03-03,INCOMPLETE,434,3176718735,EVALUATED


In [19]:
daily_engagement_pd.head()

,acct,utc_date,num_courses_visited,total_minutes_visited,lessons_completed,projects_completed
0,0,2015-01-09,1.0,11.679374,0.0,0.0
1,0,2015-01-10,2.0,37.284887,0.0,0.0
2,0,2015-01-11,2.0,53.633746,0.0,0.0
3,0,2015-01-12,1.0,33.489270,0.0,0.0
4,0,2015-01-13,1.0,64.779678,0.0,0.0



# Is my data ready to work with?
Let us look at the data types:

In [6]:
print("The data types for enrollments are:")
for key, val in enrollments[0].items():
    print(key, "\t", type(val))    

The data types for enrollments are:
account_key 	 <class 'str'>
status 	 <class 'str'>
join_date 	 <class 'str'>
cancel_date 	 <class 'str'>
days_to_cancel 	 <class 'str'>
is_udacity 	 <class 'str'>
is_canceled 	 <class 'str'>


Looks like everything just got read as a string. We have some conversion work to do! Specifically, we need to convert: (i) string to int, (ii) string to bool, (iii) string to datetime, and (iv) string to float.


In [7]:
from datetime import datetime as dt

def convert_str_to_bool(string):
    if string == "True":
        return True
    elif string == "False":
        return False
    else:
        return None
    
def convert_str_to_int(string):
    if string == "":
        return None
    else:
        return int(string)
    
def parse_date(date_str):
    if date_str == "":
        return None
    return dt.strptime(date_str, "%Y-%m-%d")
    
def convert_str_to_float(string):
    if string == "":
        return None
    else:
        return float(string)
    
    
def clean_enrollment_data_types(enrollment_data):
    for data in enrollment_data:
        data["is_canceled"] = convert_str_to_bool(data["is_canceled"])
        data["is_udacity"] = convert_str_to_bool(data["is_udacity"])
        data["join_date"] = parse_date(data["join_date"])
        data["cancel_date"] = parse_date(data["cancel_date"])
        data["days_to_cancel"] = convert_str_to_int(data["days_to_cancel"])

def clean_project_submissions_data_types(project_submissions_data):
    for data in project_submissions_data:
        data["creation_date"] = parse_date(data["creation_date"])
        data["completion_date"] = parse_date(data["completion_date"])

def clean_daily_engagement_data_types(daily_engagements_data):
    for data in daily_engagements_data:
        data["utc_date"] = parse_date(data["utc_date"]) 
        data["num_courses_visited"] = convert_str_to_float(data["num_courses_visited"])
        data["total_minutes_visited"] = convert_str_to_float(data["total_minutes_visited"])
        data["lessons_completed"] = convert_str_to_float(data["lessons_completed"])
        data["projects_completed"] = convert_str_to_float(data["projects_completed"])
        
clean_enrollment_data_types(enrollments)
clean_project_submissions_data_types(project_submissions)
clean_daily_engagement_data_types(daily_engagement)


Let's check types again:

In [8]:
print("The data types for enrollments are:")
for key, val in enrollments[0].items():
    print(key, "\t", type(val))    

The data types for enrollments are:
account_key 	 <class 'str'>
status 	 <class 'str'>
join_date 	 <class 'datetime.datetime'>
cancel_date 	 <class 'datetime.datetime'>
days_to_cancel 	 <class 'int'>
is_udacity 	 <class 'bool'>
is_canceled 	 <class 'bool'>


# Brainstorm: What can we explore?
- Now that the data is in a format we can work with, how should we start our exploration?
- As a data analyst you must develop a sense about what you can extract from the data.
- Some things that I was interested in when I looked at this dataset: 
* How does student daily engagement related to grades in projects? Is it that the higher the amount of time you spend, the better grades you get?
* How does the number of courses you are enrolled in affect your performance?
* Can we study cancellation trends, what causes students to cancel?


# Exercise 1: Data basics and inconsistencies (10 mins)

But before we get into the data, lets understand the data and if there are any inconsistencies. Is our data complete? How big is it? Is it sufficient to study these questions? 
* For each of the three files you have loaded, find the total number of rows in the csv and the number of unique students. To find the number of unique students in each table, you might want to try creating a set of the account keys. Also, try writing a function so that you do not repeat code for each table.

* Did you come across any issues while trying to write this function?
* Do you have any observations about these numbers?

####  Bonus Question: Over how much time was this data collected?

In [24]:
# Row Count Function
def row_count(dataset):
    count=0
    for i in dataset:
        count+=1        
    return count

print("Number of Rows in Enrollment Dataset: ",row_count(enrollments))
print("Number of Rows in Daily Engagement Dataset: ",row_count(daily_engagement))
print("Number of Rows in Project Submission Dataset: ",row_count(project_submissions))

# Unique Student Count Using Account Key Coloumn
def unique_count(dataset):
    x=[]
    for i in dataset:
        x.append(list(i.values())[0])
    return len(set(x))

print('Number of Unique Student Accounts in Enrollment Dataset: ',unique_count(enrollments))
print('Number of Unique Student Accounts in Daily Engagement Dataset: ',unique_count(daily_engagement))
print('Number of Unique Student Accounts in Project Submission Dataset: ',unique_count(project_submissions))

Number of Rows in Enrollment Dataset:  1640
Number of Rows in Daily Engagement Dataset:  136240
Number of Rows in Project Submission Dataset:  3642
Number of Unique Student Accounts in Enrollment Dataset:  1302
Number of Unique Student Accounts in Daily Engagement Dataset:  1237
Number of Unique Student Accounts in Project Submission Dataset:  276


# Exercise 2 (Optional): Bounds checks
* Familiarize yourself with the possible values for each field, write checks to make sure the values fall in the expected format/set. For example, does the date field always make sense? What are the possible values of the field <b>"assigned rating"</b> in the project_submissions table?

### Fix 1: Fix inconsistencies (if any)

In [25]:
# Paste and run code that I will provide here
for record in daily_engagement:
    record["account_key"] = record["acct"] 
    del record["acct"]


### Fix 2: Filter Data

In [26]:
# Paste and run code that I will provide here
# Here we filter out data from test accounts
def get_udacity_accounts(enrollment_data):
    udacity_accounts = set()
    for record in enrollment_data:
        if record["is_udacity"]:
            udacity_accounts.add(record["account_key"])
    return udacity_accounts

def remove_udacity_test_accounts(data, test_accounts):
    filtered_data = list()
    for record in data:
        if record["account_key"] not in test_accounts:
            filtered_data.append(record)
    return filtered_data

udacity_accounts = get_udacity_accounts(enrollments)
non_udacity_enrollments = remove_udacity_test_accounts(enrollments, udacity_accounts)
non_udacity_project_submissions = remove_udacity_test_accounts(project_submissions, udacity_accounts)
non_udacity_daily_engagement = remove_udacity_test_accounts(daily_engagement, udacity_accounts)

print(len(enrollments))
print(len(non_udacity_enrollments))
print(len(non_udacity_project_submissions))
print(len(non_udacity_daily_engagement))


1640
1622
3634
135656


Note: Everytime you filter data, you should understand filtering ratios. In this case, we only filtered a handful of accounts, and looks like most of the records were retained. Can you think of a reason in this case why it is possible that we might have filtered out a lot of data?

# Part 2: Data Analysis


### Goal: How does daily engagement differ for students who pass the first project vs. for those who do not?

Some questions to think about:
- Daily engagement over how much time? All time up until the project submission? Or for a fixed period of time?
- We already know that some students cancel after enrolling. How is cancellation going to affect our numbers?

Let us refine the question to: how does daily engagement **in the first week** (after enrollment) differ for students who pass their first project? Further let us limit to students that remain enrolled for at least seven days.

# Exercise 1: Slicing, Grouping, and Crunching Data (30 mins)
### Tasks:
- Create a dictionary containing only paid students that remain enrolled for at least a week --- the dictionary should map account keys to enrollment date.
- *Slice data:* For these students, extract engagement records that are **within** one week after their enrollment date (I have provided a helper function for doing the within-one-week check)
- *Group data:* Compute the following metrics per student: total number of minutes spent, total number of lessons completed, and (optional) total number of days student visited the classroom
- Write a function describe_data, that prints the mean, standard deviation, max, and min for the above metrics

#### Check whether you are on the right track:
- Task 1: The number of keys in this dictionary should be 995. Thing to check for: Is there only one enrollment record per student in the data? If you find there are multiple, how will you deal with it?
- Task 2: The number of records you should get is: 21,508
- Task 3: Brainstorm ways in which you can check whether your crunching is correct. 

In [171]:
# Your code goes here

# print(list(non_udacity_enrollments[9].keys())[2])
# dictionary
dic = {}
for record in non_udacity_enrollments:
    if (str(record["days_to_cancel"]) >= '7') or (str(record["days_to_cancel"]) == 'None'):
        dic[record["account_key"]] = record["join_date"]

for keys,values in dic.items():
    print('Acc num: %s enrollment date: %s' % (keys,values))


# Helper function to check for data within one week
def within_one_week(record_date, enrollment_date):
    time_delta = record_date - enrollment_date
    return time_delta.days < 7


Acc num: 429 enrollment date: 2015-03-10 00:00:00
Acc num: 60 enrollment date: 2015-04-01 00:00:00
Acc num: 369 enrollment date: 2014-11-10 00:00:00
Acc num: 322 enrollment date: 2015-03-10 00:00:00
Acc num: 336 enrollment date: 2014-11-10 00:00:00
Acc num: 553 enrollment date: 2014-11-10 00:00:00
Acc num: 584 enrollment date: 2015-03-10 00:00:00
Acc num: 1006 enrollment date: 2014-11-10 00:00:00
Acc num: 458 enrollment date: 2015-04-06 00:00:00
Acc num: 1058 enrollment date: 2015-03-10 00:00:00
Acc num: 206 enrollment date: 2014-11-10 00:00:00
Acc num: 45 enrollment date: 2015-03-10 00:00:00
Acc num: 604 enrollment date: 2014-11-10 00:00:00
Acc num: 204 enrollment date: 2014-11-10 00:00:00
Acc num: 51 enrollment date: 2015-03-10 00:00:00
Acc num: 568 enrollment date: 2014-11-10 00:00:00
Acc num: 130 enrollment date: 2015-04-01 00:00:00
Acc num: 44 enrollment date: 2015-03-10 00:00:00
Acc num: 198 enrollment date: 2014-11-10 00:00:00
Acc num: 368 enrollment date: 2014-11-10 00:00:00
Ac


# Validation: Do my numbers make sense?

How do I validate that my crunching is correct, and something did not get messed up along the way? 

Write your thoughts in this markdown cell. Use task 3 above as a thinking tool.

In [172]:
# Your check goes here


# Exercise 2: Splitting passing/non-passing students (15 mins)
- Write code to split passing and non-passing students for the first project. The first project refers to the following two lesson keys: ['746169184', '3176718735']. Be careful about how you check for "passing".
- Use the code you wrote above to compare the two student groups.
- Plot histograms for the metrics you computed. What do the plots tell? (You may want to revisit the plotting code in the python-tutorial).
